# OpenMC Tallies

**Learning Objectives:**

  - Understand application of filters and scores to create tallies
  - Apply tallies to an OpenMC simulation
  - Extract information from OpenMC statepoint files
  - Understand tally units and normalization
  - Plot tally results

In [1]:
import openmc
import numpy as np
from matplotlib import pyplot as plt

In this section, we'll be looking at how to extract custom information from an OpenMC simulation in what is known as a tally. A tally accumulates statistical information during the simulation about particles when they eneter regions of phase space specified on the tally. The limits of these regions are set by filters applied to the tally. Scores and nuclides can also be applied to tallies to indicate what type of information is kept about the particle (e.g. reaction types, flux, heat, etc.).

Any tally in OpenMC can be described with the following form:

$$ 
 X = \underbrace{\int d\mathbf{r} \int d\mathbf{\Omega} \int
    dE}_{\text{filters}} \underbrace{f(\mathbf{r}, \mathbf{\Omega},
    E)}_{\text{scores}} \psi (\mathbf{r}, \mathbf{\Omega}, E)
$$

where filters set the limits of integration and the scoring function, $f$ is convolved with particle information (e.g. reaction type, current material, etc.).

## Pincell Model

First we'll need a model to examine. OpenMC has a few basic models that we can use to look at tally setup. The function below generates a 2-D PWR pincell model with reflective boundary conditions on the X-Y planes. This function provides an `openmc.Model` object, which ties together materials, geometry, tallies, and settings in a single Python object with a full problem description.

To get a better idea of what this model looks like, we'll start by generating a plot and examining the materials used.

If we look at the tallies object on our pincell model, we'll see there aren't currently any custom tallies applied.

In this exercise we'll be adding tallies to perform a few different tasks:


  **1. Determine the energy and heat produced per fission** \
  **2. Plot the flux spectrum of the pincell** \
  **3. Plot reaction types based on material**
  
To do this we'll use a variety of different filters applied to different tallies. 

## Task 1: Energy released per fission

First, to determine the recoverable energy produced per fission we'll create a tally without filters to gather information on the fission reaction rate ("`fission`") and recoverable fission energy ("`kappa-fission`"). Because we want this information talllied throughout the model, a "global" tally, no filters need to be applied.



<div class="alert alert-block alert-info">
A full list of scores and their meanings can be found <a href=https://docs.openmc.org/en/stable/usersguide/tallies.html#scores >here</a>.
</div>


After adjusting the default settings for number of particles and batches on the model we'll run it and examine the data.

If we list our current directory, we see that several new files have been created as a result of this run: `summary.h5`, `tallies.out`, and `statepoint.50.h5`. The summary file contains information about the simulation's setup (geometry, materials, meshes, etc.) in an HDF5 format. The `tallies.out` file contains a text output of all user-specified tallies for the simulation.

This can be useful to quickly look at simple tally results, but isn't a great format to post-process simulation data. For that we'll look to the statepoint file.

The statepoint file contains information about simulation results including tally specifications and data. The location of this statepoint file was provided to us by the `pincell_model.run()` command.

To extract information from the statepoint file we'll create an `openmc.StatePoint` object. The `statepoint.get_tally` function will search for tallies by scores, filters, nuclides, ids, and return the closest match. Exact matches can be speficied as well.

If we print the tally objects returned, we see that they indeed match the tally specification we generated above.

<div class="alert alert-block alert-info">
<b>A quick aside on how statepoint objects interact with summary files:</b>


The `openmc.statepoint` object will read information from the `summary.h5` file if one is present, keeping that file open in the Python interpreter. The open `summary.h5` file can interfere with the initialization of subsequent OpenMC simulations. It is recommended that information be extracted from statepoints within a [context manager](https://book.pythontips.com/en/latest/context_managers.html) as we do here. Alternatively, making sure to call the `openmc.StatePoint.close` method will work also. For more details please look to the [relevant section in the user's guide](https://docs.openmc.org/en/stable/usersguide/troubleshoot.html#runtimeerror-failed-to-open-hdf5-file-with-mode-w-summary-h5).   
</div>



To compute the energy released per fission event, we can simply take the tallied energy released per fission and divide it by the fission rate.

For a water reactor with U235 as the only fissioning isotope this is about what we would expect: ~193.4 MeV! Okay, on to some new tallies with filters.

 As with most values coming out of an MC code, these values are per source-particle. In this case these units cancel out, but this will not be the case in our next example.

## Task 2: Plot the neutron flux spectrum


To perform this task, we'll be applying a tally with an energy filter and a score. OpenMC's data module contains different group structures. For this problem we'll use the CASMO-70 group structure. An energy filter can easily be created from a pre-defined group structure in OpenMC as follows:

Now we'll apply this tally and re-run the problem

Now to plot the spectrum

## Normalizing Tallies

Note that the units of flux in the above plot are in $\frac{particle-cm}{source-particle}$. As is the case with many values tallied by Monte Carlo codes, the value of the flux does not account for volume and is in terms of the number of source particles emitted. To generate this same plot in terms of absolute flux units ($\frac{particle}{cm^{2}-s}$) we'll need to normalize this tally by:

  - the volume of the region the tally covers
  - the number of source particle emitted

In this case, the volume of the region is the volume of the entire pincell. Because we're working with a 2-D model, we'll get units that give us the flux per unit length of the pincell in the axial direction. For simplicity, we'll assume that our pincell is 1 cm in height to make life easier. OpenMC's geometry object allows for computation of a bounding box for geometries like this that will help make this more straight forward.

Determining the number of source particles per second is more complicated, however. This means computing the energy produced per fission reaction. To get this value we'll need the following pieces of information:

  1. the total power produced in the tally region (known a priori)
  2. the heat produced by fission power


For fixed-source simulations, the source emission rate(s) are provided directly as a normalization factor, most often in units of $\frac{source-particles}{s}$. For our eigenvalue simuoation, we'll need to get the additional information needed in another tally.

The combination of the following tally values and power provide us with the source normalization needed as follows:


$$ \text{neutron source} [\frac{n}{s}] = \text{power} [\frac{J}{s}] \times \frac{1}{1.6\times 10^{-19}} [\frac{eV}{J}] \times \frac{1}{\text{fission heat} [\frac{eV}{source}]} $$ 

We can now use this information to normalize our flux values and reproduce our plot in more standard units.

## Task 3: Reaction Types by Material

Looking at the different reaction types by material will require a material filter and the set of reaction types we want to score. For this example, we'll be scoring absorption, scattering and fission in each material.

To start, we'll create a material filter.

Now we'll gather information from the statepoint file about each score we applied to the tally. With multiple scores and materials, we'll use a Pandas data frame to view the results in a more coherent manner.

Each score has three values -- one for each material in the model.

First, we'll add a new column to the data frame with normalized results.

We'll add a new entry in the dataframe for our material names to make plotting easier.